In [ ]:
########################### Train data with pre-trained model #####################################
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Load data from CSV
data = pd.read_csv('data.csv')

# Extract feature columns (temperature, humidity, light) and target column (isWaterNeeded)
X = data[['temperature', 'humidity', 'light']].values
y = data['isWaterNeeded'].values

# Convert each row into a 32x32x3 image
images = []
for row in X:
    image = np.full((32, 32, 3), row)
    images.append(image / 255.0)  # Normalize images then append

# Convert the list of images and labels to numpy arrays
images = np.array(images)
labels = np.array(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Load the ResNet50 model (excluding the top layer)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom layers on top of the ResNet50 model
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Freeze the pre-trained layers
base_model.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

# Make predictions
predictions = model.predict(X_test)

94765736/94765736 [==============================] - 6s 0us/step
Epoch 1/5
2500/2500 [==============================] - 38s 10ms/step - loss: 0.3512 - accuracy: 0.8442
Epoch 2/5
2500/2500 [==============================] - 23s 9ms/step - loss: 0.1803 - accuracy: 0.9214
Epoch 3/5
2500/2500 [==============================] - 24s 10ms/step - loss: 0.1587 - accuracy: 0.9304
Epoch 4/5
2500/2500 [==============================] - 24s 10ms/step - loss: 0.1499 - accuracy: 0.9338
Epoch 5/5
625/625 [==============================] - 7s 10ms/step - loss: 0.1418 - accuracy: 0.9426
Test loss: 0.1418362855911255
Test accuracy: 0.942550003528595
625/625 [==============================] - 6s 8ms/step


In [ ]:
import requests
import time

while (True):
  response = requests.get("https://api.thingspeak.com/channels/2149892/feeds.json?api_key=NOA20XYUMWR6DN6W&results=1").json()

  feeds = response['feeds']

  temperatures = []
  humidities = []

  for feed in feeds:
      temperatures.append(float(feed['field1']))
      humidities.append(float(feed['field2']))

  avg_temperature = round(sum(temperatures) / len(temperatures), 1)
  avg_humidity = round(sum(humidities) / len(humidities), 1)
  avg_light = 0.0

  sample = [avg_temperature, avg_humidity, avg_light]

  sample_image = np.full((32, 32, 3), sample)

  # Normalize the sample
  sample_image = sample_image / 255.0

  # Expand dimensions to create a batch of size 1
  sample_image = np.expand_dims(sample_image, axis=0)

  prediction = model.predict(sample_image)

  isNeedWater = 1 if prediction[0][0] > 0.5 else 0

  print("Average: [temperature, humidity, light] =", sample)
  print("Does the plant need water?", isNeedWater)

  response = requests.get("https://api.thingspeak.com/update?api_key=7U16F0PKBCATGNDB&field1=" + str(isNeedWater))

  time.sleep(5)